In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [86]:
out='../output/'
inp='../input/monthly/'
pref='idm'

In [87]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,Min,,
2,Max,,
3,Average,,
4,Total,,
...,...,...,...
648,(2022/06/27) 23-24,49.75,149
649,Min,0,0
650,Max,315,174
651,Average,19.587407407407407407407407407,10.476851851851851851851851852


In [88]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
5,(2017/05/04) 04-05,0.40,0.5
6,(2017/05/04) 05-06,0.00,0.0
7,(2017/05/04) 06-07,0.00,0.0
8,(2017/05/04) 07-08,0.00,0.0
9,(2017/05/04) 08-09,1.50,8.0
...,...,...,...
644,(2022/06/27) 19-20,315.00,50.0
645,(2022/06/27) 20-21,107.50,97.0
646,(2022/06/27) 21-22,87.50,151.0
647,(2022/06/27) 22-23,52.50,96.0


In [89]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
5,(2017/05/04) 04-05,0.40,0.5,2017-05-04,4
6,(2017/05/04) 05-06,0.00,0.0,2017-05-04,5
7,(2017/05/04) 06-07,0.00,0.0,2017-05-04,6
8,(2017/05/04) 07-08,0.00,0.0,2017-05-04,7
9,(2017/05/04) 08-09,1.50,8.0,2017-05-04,8
...,...,...,...,...,...
644,(2022/06/27) 19-20,315.00,50.0,2022-06-27,19
645,(2022/06/27) 20-21,107.50,97.0,2022-06-27,20
646,(2022/06/27) 21-22,87.50,151.0,2022-06-27,21
647,(2022/06/27) 22-23,52.50,96.0,2022-06-27,22


In [90]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
5,0.40,0.5,2017-05-04,4
6,0.00,0.0,2017-05-04,5
7,0.00,0.0,2017-05-04,6
8,0.00,0.0,2017-05-04,7
9,1.50,8.0,2017-05-04,8
...,...,...,...,...
644,315.00,50.0,2022-06-27,19
645,107.50,97.0,2022-06-27,20
646,87.50,151.0,2022-06-27,21
647,52.50,96.0,2022-06-27,22


In [91]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2017-05-04,0.40,0.5,4,2017,5,4,3
2017-05-04,0.00,0.0,5,2017,5,4,3
2017-05-04,0.00,0.0,6,2017,5,4,3
2017-05-04,0.00,0.0,7,2017,5,4,3
2017-05-04,1.50,8.0,8,2017,5,4,3
...,...,...,...,...,...,...,...
2022-06-27,315.00,50.0,19,2022,6,27,0
2022-06-27,107.50,97.0,20,2022,6,27,0
2022-06-27,87.50,151.0,21,2022,6,27,0


In [92]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=hourly.loc[hourly['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=hourly.loc[hourly['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=hourly.loc[(hourly['hour']>=12) & (hourly['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()
daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2017 5     4      41.8      57.710         1.380622       1.50        0.0   
           5     117.1     167.963         1.434355       2.00        0.0   
           6      82.8      56.174         0.678430       0.90        0.0   
           7      18.3       8.418         0.460000       0.46        0.0   
           8      69.7     104.994         1.506370       1.92        0.0   
...                ...         ...              ...        ...        ...   
2022 6     23    183.0   24250.000       132.513661     200.00        0.0   
           24    240.0   29200.000       121.666667     200.00        0.0   
           25     32.0     640.000        20.000000      20.00        0.0   
           26      0.0       0.000              NaN       0.00        0.0   
           27   1412.0  151744.420       107.467720     315.00        0.0   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2017 5     4      0.635000        8.0        0.0    2.090000   
           5      0.884583       15.5        0.0    4.879167   
           6      0.396667       11.8        0.0    3.450000   
           7      0.076667        6.1        0.0    0.762500   
           8      0.647500       13.0        0.0    2.904167   
...                    ...        ...        ...         ...   
2022 6     23    13.229167       87.0        0.0    7.625000   
           24    27.083333       80.0        0.0   10.000000   
           25     5.000000        7.0        0.0    1.333333   
           26     0.000000        0.0        0.0    0.000000   
           27    75.926667      174.0        0.0   58.833333   

                price_morning_max  price_evening_max  price_standard  \
year month day                                                         
2017 5     4                 1.50               1.50          1.3000   
           5                 1.64               2.00          1.3000   
           6                 0.70               0.90          0.1150   
           7                 0.00               0.46          0.0000   
           8                 1.30               1.92          1.3000   
...                           ...                ...             ...   
2022 6     23              100.00             200.00          0.0000   
           24              200.00             200.00          0.0000   
           25               20.00               0.00          0.0000   
           26                0.00               0.00          0.0000   
           27               85.00             315.00         87.4975   

                     date  
year month day             
2017 5     4   2017-05-04  
           5   2017-05-05  
           6   2017-05-06  
           7   2017-05-07  
           8   2017-05-08  
...                   ...  
2022 6     23  2022-06-23  
           24  2022-06-24  
           25  2022-06-25  
           26  2022-06-26  
           27  2022-06-27  

[1873 rows x 13 columns]

In [93]:
hourly['cat']='off'

if (pref=='dam'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2017-05-04,0.40,0.5,4,2017,5,4,3,0.20,off
2017-05-04,0.00,0.0,5,2017,5,4,3,0.00,off
2017-05-04,0.00,0.0,6,2017,5,4,3,0.00,off
2017-05-04,0.00,0.0,7,2017,5,4,3,0.00,off
2017-05-04,1.50,8.0,8,2017,5,4,3,12.00,off
...,...,...,...,...,...,...,...,...,...
2022-06-27,315.00,50.0,19,2022,6,27,0,15750.00,off
2022-06-27,107.50,97.0,20,2022,6,27,0,10427.50,off
2022-06-27,87.50,151.0,21,2022,6,27,0,13212.50,off


In [94]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2017 5     4      41.8      57.710         1.380622       1.50        0.0   
           5     117.1     167.963         1.434355       2.00        0.0   
           6      82.8      56.174         0.678430       0.90        0.0   
           7      18.3       8.418         0.460000       0.46        0.0   
           8      69.7     104.994         1.506370       1.92        0.0   
...                ...         ...              ...        ...        ...   
2022 6     23    183.0   24250.000       132.513661     200.00        0.0   
           24    240.0   29200.000       121.666667     200.00        0.0   
           25     32.0     640.000        20.000000      20.00        0.0   
           26      0.0       0.000              NaN       0.00        0.0   
           27   1412.0  151744.420       107.467720     315.00        0.0   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2017 5     4      0.635000        8.0        0.0    2.090000   
           5      0.884583       15.5        0.0    4.879167   
           6      0.396667       11.8        0.0    3.450000   
           7      0.076667        6.1        0.0    0.762500   
           8      0.647500       13.0        0.0    2.904167   
...                    ...        ...        ...         ...   
2022 6     23    13.229167       87.0        0.0    7.625000   
           24    27.083333       80.0        0.0   10.000000   
           25     5.000000        7.0        0.0    1.333333   
           26     0.000000        0.0        0.0    0.000000   
           27    75.926667      174.0        0.0   58.833333   

                price_morning_max  ...  morning_hour_count  \
year month day                     ...                       
2017 5     4                 1.50  ...                 NaN   
           5                 1.64  ...                 NaN   
           6                 0.70  ...                 NaN   
           7                 0.00  ...                 NaN   
           8                 1.30  ...                 NaN   
...                           ...  ...                 ...   
2022 6     23              100.00  ...                 NaN   
           24              200.00  ...                 NaN   
           25               20.00  ...                 NaN   
           26                0.00  ...                 NaN   
           27               85.00  ...                 NaN   

                evening_hour_count off_volume  standard_volume  \
year month day                                                   
2017 5     4                   NaN       41.8              NaN   
           5                   NaN      117.1              NaN   
           6                   NaN       82.8              NaN   
           7                   NaN       18.3              NaN   
           8                   NaN       69.7              NaN   
...                            ...        ...              ...   
2022 6     23                  NaN      183.0              NaN   
           24                  NaN      240.0              NaN   
           25                  NaN       32.0              NaN   
           26                  NaN        0.0              NaN   
           27                  NaN     1412.0              NaN   

                morning_volume  evening_volume   off_value  standard_value  \
year month day                                                               
2017 5     4               NaN             NaN      57.710             NaN   
           5               NaN             NaN     167.963             NaN   
           6               NaN             NaN      56.174             NaN   
           7               NaN             NaN       8.418             NaN   
           8               NaN             NaN 

In [95]:
daily.to_csv(out + pref + '_daily.csv')

In [96]:
hourly.to_csv(out + pref + '_hourly.csv')

In [97]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2017-05-04,4,0.40,0.5,off
2017-05-04,5,0.00,0.0,off
2017-05-04,6,0.00,0.0,off
2017-05-04,7,0.00,0.0,off
2017-05-04,8,1.50,8.0,off
...,...,...,...,...
2022-06-27,19,315.00,50.0,off
2022-06-27,20,107.50,97.0,off
2022-06-27,21,87.50,151.0,off


In [98]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [99]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,5,133.964286,169.218714,1.263000,0.0,0.761149,15.625000,0.235714,5.680149,...,0.0,1.263163,1.263163,NaN,NaN,NaN,5.683333,NaN,NaN,NaN
1,2017,6,69.990000,124.529467,1.224500,0.0,0.669617,15.276667,0.000000,3.049222,...,0.0,1.779247,1.779247,NaN,NaN,NaN,2.999571,NaN,NaN,NaN
2,2017,7,65.567742,84.076677,1.192083,0.0,0.541298,11.129032,0.016129,2.815511,...,0.0,1.282287,1.282287,NaN,NaN,NaN,2.823056,NaN,NaN,NaN
3,2017,8,52.654839,68.060452,1.337500,0.0,0.481457,10.132258,0.170968,2.306425,...,0.0,1.292577,1.292577,NaN,NaN,NaN,2.267083,NaN,NaN,NaN
4,2017,9,73.586667,79.745200,0.987083,0.0,0.431506,11.293333,0.000000,3.164722,...,0.0,1.083691,1.083691,NaN,NaN,NaN,3.153714,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2022,2,371.285714,20549.907143,46.150000,0.0,12.127009,59.964286,0.000000,15.820833,...,0.0,55.347961,55.347961,NaN,NaN,NaN,15.751515,NaN,NaN,NaN
58,2022,3,377.483871,17052.145161,45.170000,0.0,12.922285,41.000000,2.612903,17.338710,...,0.0,45.173176,45.173176,NaN,NaN,NaN,16.252778,NaN,NaN,NaN
59,2022,4,295.517241,16206.275862,46.416500,0.0,10.263629,49.137931,2.241379,12.918103,...,0.0,54.840373,54.840373,NaN,NaN,NaN,12.602941,NaN,NaN,NaN
60,2022,5,278.580000,16302.191200,51.451250,0.0,12.128117,47.960000,0.000000,11.615500,...,0.0,58.518886,58.518886,NaN,NaN,NaN,11.685403,NaN,NaN,NaN


In [100]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,5,133.964286,169.218714,1.263000,0.0,0.761149,15.625000,0.235714,5.680149,...,0.0,1.263163,1.263163,NaN,NaN,NaN,5.683333,NaN,NaN,NaN
1,2017,6,69.990000,124.529467,1.224500,0.0,0.669617,15.276667,0.000000,3.049222,...,0.0,1.779247,1.779247,NaN,NaN,NaN,2.999571,NaN,NaN,NaN
2,2017,7,65.567742,84.076677,1.192083,0.0,0.541298,11.129032,0.016129,2.815511,...,0.0,1.282287,1.282287,NaN,NaN,NaN,2.823056,NaN,NaN,NaN
3,2017,8,52.654839,68.060452,1.337500,0.0,0.481457,10.132258,0.170968,2.306425,...,0.0,1.292577,1.292577,NaN,NaN,NaN,2.267083,NaN,NaN,NaN
4,2017,9,73.586667,79.745200,0.987083,0.0,0.431506,11.293333,0.000000,3.164722,...,0.0,1.083691,1.083691,NaN,NaN,NaN,3.153714,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2022,2,371.285714,20549.907143,46.150000,0.0,12.127009,59.964286,0.000000,15.820833,...,0.0,55.347961,55.347961,NaN,NaN,NaN,15.751515,NaN,NaN,NaN
58,2022,3,377.483871,17052.145161,45.170000,0.0,12.922285,41.000000,2.612903,17.338710,...,0.0,45.173176,45.173176,NaN,NaN,NaN,16.252778,NaN,NaN,NaN
59,2022,4,295.517241,16206.275862,46.416500,0.0,10.263629,49.137931,2.241379,12.918103,...,0.0,54.840373,54.840373,NaN,NaN,NaN,12.602941,NaN,NaN,NaN
60,2022,5,278.580000,16302.191200,51.451250,0.0,12.128117,47.960000,0.000000,11.615500,...,0.0,58.518886,58.518886,NaN,NaN,NaN,11.685403,NaN,NaN,NaN


In [101]:
monthly=monthly.set_index(['year','month'])

In [102]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [103]:
monthly.to_csv(out + pref + '_monthly.csv')

In [104]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [105]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,price_morning_max,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,25433.202497,37.760627,42.035742,0.250581,16.628492,65.063871,0.216774,21.547871,34.357355,...,0.0,788429.277400,720.000000,0.0,0.0,0.0,21.659944,NaN,NaN,NaN
1,2,12518.608214,35.423741,34.499214,0.171429,10.155742,39.125714,0.142857,9.924327,23.456000,...,0.0,350521.030000,660.000000,0.0,0.0,0.0,9.770395,NaN,NaN,NaN
2,3,14070.079277,34.008333,34.560129,0.462323,11.500721,39.643871,0.792903,12.168435,26.838968,...,0.0,436172.457600,720.000000,0.0,0.0,0.0,11.812861,NaN,NaN,NaN
3,4,18848.786342,43.578336,51.300067,0.698456,15.382975,56.203356,1.165772,15.969855,31.365906,...,0.0,561693.833000,696.000000,0.0,0.0,0.0,15.720360,NaN,NaN,NaN
4,5,11892.330825,37.403109,46.099718,0.279661,9.794871,47.005650,0.758192,10.764120,23.737797,...,0.0,350823.759333,689.333333,0.0,0.0,0.0,10.683285,NaN,NaN,NaN
5,6,27858.799203,55.230630,84.127006,1.635198,22.659882,57.402260,0.875706,14.171469,46.497232,...,0.0,821834.576500,690.000000,0.0,0.0,0.0,14.104365,NaN,NaN,NaN
6,7,28119.714200,39.562673,66.078323,0.015871,19.715186,73.490968,0.017419,19.638118,50.210645,...,0.0,871711.140200,720.000000,0.0,0.0,0.0,19.550778,NaN,NaN,NaN
7,8,22140.519290,51.247595,69.030258,0.542323,16.860625,49.944516,0.582581,14.388371,39.230839,...,0.0,686356.098000,720.000000,0.0,0.0,0.0,14.543639,NaN,NaN,NaN
8,9,17198.072747,30.850385,36.370333,0.456733,11.993442,49.975333,0.242000,16.159550,25.848800,...,0.0,515942.182400,700.000000,0.0,0.0,0.0,16.153400,NaN,NaN,NaN
9,10,26036.634123,38.199695,51.913806,1.211548,16.278344,68.813548,0.751613,22.520059,33.486516,...,0.0,807135.657800,720.000000,0.0,0.0,0.0,22.711278,NaN,NaN,NaN


In [106]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [107]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price     volume        value
month hour                                   
1     0      6.267280   7.562400   229.289928
      1      7.354080   8.378400   268.637128
      2      7.864960  10.994400   354.987208
      3      8.104240  12.470400   406.189688
      4      8.334903  14.364516   457.878148
...               ...        ...          ...
12    19    27.068000  11.377419   974.025697
      20    20.872581  13.998710  1216.757116
      21    12.206774  12.587742   643.392839
      22     4.075548   5.479355   146.764123
      23     6.085419  10.083226   302.208942

[288 rows x 3 columns]

In [108]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [109]:
yearly_calmonthly_hours=hourly.groupby(['year','month','hour']).mean()
yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)
yearly_calmonthly_hours

price     volume        value
year month hour                                   
2017 5     0      0.244000   1.000000     0.547840
           1      0.212800   0.896000     0.479520
           2      0.205600   0.868000     0.455520
           3      0.276800   1.168000     0.617440
           4      0.343214   2.917857     1.696500
...                    ...        ...          ...
2022 6     19    27.500000  13.592593  1456.759259
           20    19.314815  17.703704  1223.166667
           21    23.833333  15.333333   976.944444
           22     3.277778   5.222222   213.888889
           23     7.861111   9.777778   355.101852

[1488 rows x 3 columns]

In [110]:
yearly_calmonthly_hours.to_csv(out + pref + '_yearly_calmonthly_hours.csv')

In [111]:
from statsmodels.tsa.stattools import pacf

In [112]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.0,1.0,1.0,1.000000
1,NaN,NaN,NaN,0.722480
2,NaN,NaN,NaN,0.385657
3,NaN,NaN,NaN,0.017218
4,NaN,NaN,NaN,0.034449
5,NaN,NaN,NaN,0.178935
6,NaN,NaN,NaN,-0.005483
7,NaN,NaN,NaN,0.018229
8,NaN,NaN,NaN,-0.121574
9,NaN,NaN,NaN,0.074477


In [113]:
pacfs.to_csv(out + pref + '_pacfs.csv')